# CHEQUEOS POST LIQUIDACION

#### Obtencion de Datos

### Importaciones

In [1]:
from decouple import config
import pyodbc as mssql
import numpy as np
import pandas as pd
from datetime import datetime

%config IPCompleter.greedy=True
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from IPython.display import Markdown as md
import xlsxwriter as xw

import warnings
warnings.filterwarnings('ignore')


#### Parámetros para el cálculo

In [ ]:
LIQUIDACION = config("LIQUIDACION") #, default="{ye:0000}-{mo:0>2}".format(ye=datetime.now().year, mo=datetime.now().month)) 
MINIMO_GENERAL = config("MINIMO_GENERAL", 0.0, cast=float)
MINIMO_POLICIAS = config("MINIMO_POLICIAS", 0.0, cast=float)
PROMEDIO_INCREMENTOS = config("INCREMENTO_MEDIO", default=0.1, cast=float)
EXCLUIR_REPARTICIONES_MINIMO_GARANTIZADO = [17, 26]

# codigos_bruto = ['0Q','0R','0N','0S','LC','OC','PC','QC','TC','U0','MC','PA','TE','XX','PX','MX','YX','NX','1X']



In [3]:
# archivo de salida
writer = pd.ExcelWriter("resultados/Resumen_actual_contra_" + LIQUIDACION + ".xlsx", "xlsxwriter")
wb = writer.book

formato_moneda = wb.add_format({'num_format': '$ #,##0.00'})
formato_porcentaje = wb.add_format({'num_format': '#,##0.00 %'})
formato_fecha = wb.add_format({'num_format': 'dd/mm/yyyy'})

In [4]:
SERVER = config("SERVIDOR", "IPSBD") #"IPSBD"
conn = mssql.connect(f"DRIVER={{SQL SERVER}};SERVER={SERVER};DATABASE=LIQUIDACION;;Trusted_Connection=Yes;")

#### Obtención de la fecha de alta de la liquidacion seleccionada

In [5]:
from datetime import datetime
altas_desde = ''
altas_desde = pd.read_sql(f"""SELECT create_date FROM SYS.DATABASES
WHERE NAME like '%{LIQUIDACION}%'""",conn)["create_date"][0].strftime('%d/%m/%Y')

#### Obtención de Datos Generales

In [6]:
codigos = pd.read_sql( "SELECT * FROM CODIGOS", conn, index_col="CODIGO")
reparticion = pd.read_sql( "SELECT * FROM REPARTICION", conn, index_col="Cod" )
tipos_beneficio = pd.read_sql ( "SELECT * FROM BENEFICIO", conn, index_col="Cod" )
codigos_bruto_minimo = pd.read_sql("SELECT CODIGO FROM CODIGOS WHERE TIPO='C' AND BRUTO_MINIMO=1", conn)
codigos_minimo = codigos_bruto_minimo["CODIGO"]

### 
### Auditorias desde la fecha de creacion de la liquidacion de referencia
### para ser excluidas de la comparativa, por ser casos de "desvio" a los generales
### 
auditorias = pd.read_sql( """SELECT CONTROL FROM AUDIT A WHERE A.ORIGEN='D' AND 
A.FECHA > CONVERT(int, CONVERT(datetime, '{altas_desde}'))+2""".format(altas_desde=altas_desde), conn, index_col="CONTROL")

beneficios = pd.read_sql( """SELECT CONTROL, PLLA, ORDEN, AFILIADO, BENEFICIO, COD_BENEFICIO, LETRA,
CASE WHEN COD_BENEFICIO in (7,25,27,29) then DBO.PORCENTAJE_COMPARTIDA(CONTROL) ELSE 1 END AS PORCENTAJE
FROM Liquidacion.dbo.DatosAfiliado
WHERE HABILITADO='S'""", conn, index_col="CONTROL")
beneficios[["COD_BENEFICIO"]]=beneficios[["COD_BENEFICIO"]].astype("int64")

### Excluímos las auditorias de altas y modificaciones para evitar las "deformaciones"
beneficios_sin_altas = beneficios[~beneficios.index.isin(auditorias.index)]

liquidacion_ben = pd.read_sql(""" SELECT * FROM LIQUIDACION.DBO.LIQUIDACION""", conn, index_col=["PLLA", "ORDEN", "AFILIADO"])
beneficios_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.DATOSAFILIADO".format(liquidacion=LIQUIDACION), \
                             conn, index_col=["CONTROL"])
liquidacion_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.LIQUIDACION".format(liquidacion=LIQUIDACION), \
                                conn, index_col=["PLLA", "ORDEN", "AFILIADO"])



### Determinacion de brutos

#### Determinacion de los brutos actuales

In [7]:
### determinacion de la tabla de brutos de liquidacion activa
brutos_act = beneficios_sin_altas.reset_index().merge(liquidacion_ben, left_on=["PLLA", "ORDEN", "AFILIADO"],\
                              right_on=["PLLA", "ORDEN", "AFILIADO"])
brutos_act = brutos_act[brutos_act["CODIGO"].isin(codigos_bruto)].groupby(["CONTROL"]).sum("MONTO")[["MONTO"]]
# agrego aquellos que no tienen codigos de bruto
brutos_act = brutos_act.join(beneficios_sin_altas,  how="outer")
brutos_act["MONTO"].fillna(value=0.0, inplace=True)
brutos_act["BRUTO"] = brutos_act["MONTO"] + brutos_act["BENEFICIO"]
# dejo solo la columna del bruto recien calculada
brutos_act=brutos_act[["BRUTO","BENEFICIO", "COD_BENEFICIO", "LETRA"]]

#### Determinación de los brutos de referencia

In [8]:
###determinacion de la tabla de brutos de la liquidacion de referencia 
brutos_liq = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")
#brutos_liq = brutos_liq[brutos_liq["CODIGO"].isin(codigos_bruto)].groupby(by="CONTROL").sum()[["MONTO"]]

brutos_liq.drop(columns=["CONTROL"], axis=1, inplace=True)
brutos_liq = brutos_liq[brutos_liq["CODIGO"].isin(codigos_bruto)].groupby(by="CONTROL").sum("MONTO")


In [9]:
brutos_liq = brutos_liq.join(beneficios_liq, on="CONTROL", how="outer", lsuffix="br_l" )
brutos_liq["MONTO"].fillna(value=0.0, inplace=True)
brutos_liq["BRUTO"] = brutos_liq["MONTO"] + brutos_liq["BENEFICIO"]

brutos_liq=brutos_liq[["CONTROL", "BRUTO", "BENEFICIO"]]
brutos_liq.set_index(keys="CONTROL", inplace=True)
brutos_liq=brutos_liq.sort_values(by="CONTROL")

### Comparativa de Brutos y Beneficios

In [10]:
###  comparativa 

brutos = brutos_liq.join( brutos_act, lsuffix="_liq", rsuffix="_act" )
brutos["DIF_BRUTOS"] = (brutos["BRUTO_act"]-brutos["BRUTO_liq"])/brutos["BRUTO_liq"]
brutos["DIF_BENEFICIOS"] = (brutos["BENEFICIO_act"]-brutos["BENEFICIO_liq"])/brutos["BENEFICIO_liq"]
diferencia_brutos=brutos.loc[(brutos["DIF_BRUTOS"] > 0.0) | \
                             (brutos["DIF_BENEFICIOS"] > 0.0), ["LETRA", "COD_BENEFICIO", "DIF_BRUTOS","DIF_BENEFICIOS", ]] \
                            .sort_values(by="DIF_BRUTOS", ascending=False)
diferencia_brutos=diferencia_brutos.astype({ 'LETRA' : 'int32', 'COD_BENEFICIO' : 'int32'})
diferencia_brutos=diferencia_brutos.join(reparticion, on="LETRA", how="inner", lsuffix=".db", rsuffix=".rep")
diferencia_brutos=diferencia_brutos.join(tipos_beneficio, on="COD_BENEFICIO", how="inner", lsuffix=".db", rsuffix=".tb")
diferencia_brutos=diferencia_brutos[["Detalle.db", "Detalle.tb", "DIF_BRUTOS", "DIF_BENEFICIOS" ]] \
    .rename(columns={"Detalle.db":"Reparticion", "Detalle.tb":"Tipo_Beneficio"})
diferencia_brutos=diferencia_brutos.sort_values(["Reparticion", "Tipo_Beneficio"])

diferencia_brutos.to_excel(writer, "Diferencia Brutos",startrow=0, header=True, index=True)


(max_row, max_col)= diferencia_brutos.shape
column_settings = []
column_settings = np.append(column_settings, {'header': "CONTROL"})
column_settings = np.append(column_settings, [{'header': column} for column in diferencia_brutos.columns])

worksheet = writer.sheets["Diferencia Brutos"]
worksheet.add_table(0, 0, max_row+1, max_col, {'columns': column_settings, 'autofilter': False, \
                                             'banded_rows': False, "style": None})

worksheet.set_column("A:A", 8)
worksheet.set_column("B:C", 35)
worksheet.set_column("D:E", 15, formato_porcentaje )



0

#### Diferencia por reparticion

In [11]:
### POR REPARTICION ###

inf_rep_letra = brutos.merge(reparticion, left_on="LETRA", right_on="Cod", how="inner")
inf_rep_letra = inf_rep_letra[['Detalle', 'BRUTO_liq', 'BENEFICIO_liq', 'BRUTO_act', 'BENEFICIO_act']].groupby(by="Detalle")\
                    .agg({'Detalle':'count', 'BRUTO_liq': 'sum', "BRUTO_act": "sum", "BENEFICIO_liq": "sum", "BENEFICIO_act": "sum"})
inf_rep_letra["DIF_BRUTOS"] = (inf_rep_letra["BRUTO_act"]-inf_rep_letra["BRUTO_liq"])/inf_rep_letra["BRUTO_liq"]
inf_rep_letra["DIF_BENEFICIOS"] = (inf_rep_letra["BENEFICIO_act"] - inf_rep_letra["BENEFICIO_liq"])/inf_rep_letra["BENEFICIO_liq"]
inf_rep_letra=inf_rep_letra[(inf_rep_letra["DIF_BRUTOS"]>0.01) | (inf_rep_letra["DIF_BENEFICIOS"]>0.01)]
inf_rep_letra=inf_rep_letra.rename(columns={"Detalle":"Cantidad"})

inf_rep_letra.to_excel(writer, "Dif Brutos Por Rep")

(max_row, max_col)= inf_rep_letra.shape

column_settings = []
column_settings = np.append(column_settings, {'header': "Reparticion"})
column_settings = np.append(column_settings, [{'header': column} 
    for column in  inf_rep_letra.columns])

worksheet = writer.sheets["Dif Brutos Por Rep"]
worksheet.add_table(0, 0, max_row, max_col, {'columns': column_settings, 
                                        'autofilter': False, \
                                        'banded_rows': False, "style": None})

worksheet.set_column("A:A", 35)
worksheet.set_column("B:B", 10)
worksheet.set_column("C:F", 20, formato_moneda)
worksheet.set_column("G:H", 20, formato_porcentaje )



0

### Mínimos

#### Mínimos Administración Central y Organismos

In [12]:
brutos_minimo = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

,PLLA,ORDEN,AFILIADO,BENEFICIO,LIQUIDO,CECACOR,LECOP,LIQUIDO1,LETRA,COD_BENEFICIO,CODLOCALIDAD
CONTROL,,,,,,,,,,,
1,001,0001,82989,859626.06,1004889.59,0.00,0.00,1004889.59,6,1,8
2,001,0002,86230,1115195.80,1199539.85,0.00,0.00,1199539.85,6,1,8
39551,001,0003,58982,369486.98,479930.84,0.00,0.00,479930.84,6,1,8
20804,001,0006,91504,693821.10,1003865.16,0.00,0.00,1003865.16,6,1,8
7,001,0008,86486,1339307.63,1407655.36,0.00,0.00,1407655.36,6,1,8
...,...,...,...,...,...,...,...,...,...,...,...
19378,082,0002,81847,3926923.90,3705336.89,0.00,0.00,3705336.89,25,2,4
19381,082,0005,81895,3052150.37,2966951.36,0.00,0.00,2966951.36,25,2,4
19382,082,0007,81904,3653875.88,3336479.91,0.00,0.00,3354479.91,25,2,4


In [18]:
brutos_minimo = brutos_minimo[brutos_minimo["CODIGO"].isin(codigos_minimo)].groupby(by="CONTROL").sum()[["MONTO"]]


ValueError: 'CONTROL' is both an index level and a column label, which is ambiguous.

In [ ]:

brutos_liq
brutos_minimo.drop(columns=["CONTROL"], axis=1, inplace=True)
brutos_minimo = brutos_liq[brutos_liq["CODIGO"].isin(codigos_bruto)].groupby(by="CONTROL").sum("MONTO")
brutos_minimo


In [ ]:

minimos["ZX_act"] = np.where((~minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_general*minimos["PORCENTAJE"])), minimo_general*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
minimos.fillna(0, inplace=True)

minimos.loc[(minimos["BRUTO_act"]+minimos["ZX_act"]<(MINIMO_GENERAL*minimos["PORCENTAJE"])-0.1)&(minimos["ZX_act"]>0),["BRUTO_act", "ZX_act"]].to_excel(writer, "Minimos_en_defecto")

minimos.loc[(minimos["BRUTO_act"]+minimos["ZX_act"]>(MINIMO_GENERAL*minimos["PORCENTAJE"])+0.1)&(minimos["ZX_act"]>0),["BRUTO_act", "ZX_act"]].to_excel(writer, "Minimos_en_exceso")

#### Mínimos Retiros Policiales y Carcelarios

In [ ]:
#minimos[(minimos["Minimo"] >= 2500) & (minimos["Minimo"] <= 2502.5)]

In [ ]:
# minimos = beneficios.join( beneficios_liq, on="CONTROL", lsuffix=".BEN", rsuffix=".BENLIQ" )\
#             .join(reparticion, on="LETRA" , rsuffix=".REP", )\
#             .join(brutos, on="CONTROL", rsuffix=".BRT")\
#             .join(tipos_beneficio, on="COD_BENEFICIO", rsuffix=".tb")
            

# minimos[minimos["BRUTO_act"] < minimo_policias]
# minimos["PX_act"] = np.where((minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_policias*minimos["PORCENTAJE"])), minimo_policias*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
# minimos.fillna(0, inplace=True)

# minimos_pagados = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
#         .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

# minimos_pagados=minimos_pagados.loc[minimos_pagados["CODIGO"].isin(['PX']), ["MONTO"]]\
#         .groupby("CONTROL").sum("MONTO")

# minimos=minimos.join(minimos_pagados, on="CONTROL", rsuffix=".MINPAG").rename(columns={"MONTO":"PX_liq", "Detalle":"Reparticion", "Detalle.tb":"TIPO_BEN"})
# minimos.fillna(0.0, inplace=True)

# minimos=minimos.loc[(np.abs(minimos["PX_act"]-minimos["PX_liq"])>1)  ,["Reparticion", "TIPO_BEN", "BRUTO_act", "PX_act", "BRUTO_liq", "PX_liq"]]

# #minimos.to_excel(writer, "Minimos Retiros Policiales")



## Retroactivos


In [ ]:
calculos = pd.read_sql("""SELECT D.CONTROL, D.NOMBRE, R.DETALLE AS REPARTICINO, DC.VALOR CUOTA, DC.FINI RETRO_DESDE, 
            DC.FFIN RETRO_HASTA, APLICARDESDE, APLICARHASTA
            FROM DATOSAFILIADO D INNER JOIN
            CALCULOS C ON C.CONTROL=D.CONTROL
            INNER JOIN DATCALCULOS DC ON DC.CODLIQ=C.CODLIQ
            INNER JOIN REPARTICION R ON R.COD=D.LETRA
            INNER JOIN PLANES P ON P.CODLIQ=C.CODLIQ
            WHERE MONTH(CONVERT(DATETIME, C.FECHA+2))=MONTH(GETDATE()) AND YEAR(CONVERT(DATETIME, C.FECHA+2))=YEAR(GETDATE())
            AND USUARIO = 'SISTEMAS'""", conn)

calculos.to_excel(writer, "Retroactivos_incrementos")

ws = writer.sheets["Retroactivos_incrementos"]
ws.set_column("E:H", 15, formato_fecha )


0

## Cerrar archivo excel y guardar

In [ ]:
### Guardado y cierre del archivo excel
writer.close()

In [ ]:
calculos

,CONTROL,NOMBRE,REPARTICINO,CUOTA,RETRO_DESDE,RETRO_HASTA,APLICARDESDE,APLICARHASTA
0,11805,SUC.GOMEZ ANTONIO ABEL,Munic. Mercedes,128032.90,45566,45689,45717,45717
1,11805,SUC.GOMEZ ANTONIO ABEL,Munic. Mercedes,89924.77,45597,45689,45717,45717
2,11805,SUC.GOMEZ ANTONIO ABEL,Munic. Mercedes,55444.54,45689,45689,45717,45717
3,12194,SUC.FERRARI RICARDO RAFAE,Munic. Mercedes,126487.88,45566,45689,45717,45717
4,12194,SUC.FERRARI RICARDO RAFAE,Munic. Mercedes,88839.61,45597,45689,45717,45717
5,12194,SUC.FERRARI RICARDO RAFAE,Munic. Mercedes,54775.47,45689,45689,45717,45717
6,28523,"SUC.RODRIGUEZ, OMAR JOSE",Munic. San Luis del Palmar,40398.87,45689,45689,45717,45717
7,35623,"BARRIOS, JOSE AURELIO",Munic. San Luis del Palmar,75801.11,45689,45689,45717,45717
8,39452,"SUC.IBARROLA, HECTOR",Munic. Mercedes,448154.63,45566,45689,45717,45717
9,39452,"SUC.IBARROLA, HECTOR",Munic. Mercedes,314764.40,45597,45689,45717,45717
